<a href="https://colab.research.google.com/github/dasom222g/learn-LLM/blob/main/02_8_%E1%84%89%E1%85%A5%E1%84%87%E1%85%B3%E1%84%8B%E1%85%AF%E1%84%83%E1%85%B3_%E1%84%90%E1%85%A9%E1%84%8F%E1%85%B3%E1%86%AB%E1%84%92%E1%85%AA_%E1%84%87%E1%85%B5%E1%84%80%E1%85%AD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. 서브워드 토큰화 개요

서브워드 토큰화(Subword Tokenization)는 단어보다 작은 단위로 텍스트를 분할하는 방법입니다. 이는 미등록 단어(OOV) 문제를 해결하고 어휘 크기를 효율적으로 관리하는 데 도움이 됩니다.

### 서브워드 토큰화의 장점:
- 어휘 크기 축소
- 미등록 단어 처리 가능
- 형태적으로 유사한 단어 간 정보 공유
- 다국어 처리에 효과적

형태소 분석과 서브워드 토큰화 차이점

다음 단어들이 있을 때, 형태소 분석과 서브워드 토큰화의 차이점이 가장 잘 드러날 것 같은 예시를 3개 선택하고 이유를 설명하세요.

```
'인공지능', '비정규직', '자연어처리', '미세먼지', '코로나바이러스', '불가사리', '레이어드'
```


가장 차이가 잘 드러나는 예시:

1. **코로나바이러스**:
   - 형태소 분석: '코로나' + '바이러스'
   - 서브워드: '코로', '나', '바이', '러스'와 같이 더 작은 단위로 분할 가능

2. **레이어드**:
   - 형태소 분석: 외래어로 분석하기 어려울 수 있음
   - 서브워드: '레이', '어드'와 같이 자동으로 분할 가능

3. **자연어처리**:
   - 형태소 분석: '자연어' + '처리'
   - 서브워드: '자연', '어', '처리'로 분할 가능

이유: 형태소 분석은 언어학적 규칙에 따라 의미 단위로 분할하지만, 서브워드 토큰화는 데이터 기반으로 빈도나 우도를 고려하여 분할합니다. 특히 신조어, 외래어, 복합어에서 성능이 확성될수 있다.

## 주요 토큰화 알고리즘 비교
### 1. **BPE (Byte Pair Encoding)**
BPE는 텍스트에서 가장 자주 등장하는 문자 쌍을 새로운 단일 토큰으로 병합
- 단계별 작동 원리
초기 상태
예시 문장들:



```
"low"
"lowest"
"newer"
"wider"
```



1단계: 문자 레벨 토큰화
각 단어를 문자 단위로 분해
```
"low" → l o w
"lowest" → l o w e s t
"newer" → n e w e r
"wider" → w i d e r
```
2단계: 문자 쌍 빈도 계산
가장 빈번한 문자 쌍 찾기
복사빈도 계산:
```
'lo': 2회
'ow': 2회
'we': 2회
'er': 2회
```
3단계: 가장 빈번한 쌍 병합
예: 'lo' 병합
```
변경 전: l o w
변경 후: lo w
```
4단계: 반복 병합
계속해서 가장 빈번한 쌍 병합


### WordPiece
- BPE와 유사하지만 병합 결정 기준이 빈도가 아닌 우도(확률)(likelihood)
- BERT 및 관련 모델에서 사용
- 하위 단어는 '##'으로 시작하는 접두사로 표시

### SentencePiece
- BPE 알고리즘을 개선 및 응용한 모델
- 언어에 구애받지 않는 토큰화
- 공백을 일반 문자로 취급하는 "문장 수준" 토큰화
- 공백을 '_'(언더스코어)로 표시
- T5, LLaMA 등에서 사용
- LLM기반

In [39]:
# 실습: 다양한 토크나이저 비교해보기

import numpy as np
import re
from collections import Counter, defaultdict
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, BertTokenizer, GPT2Tokenizer, T5Tokenizer
from transformers import AutoModel

# model = AutoModel.from_pretrained("klue/bert-base")
# klue_tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")


def load_tokenizers():
    tokenizers = {
        'BERT': AutoTokenizer.from_pretrained('bert-base-uncased'), # WordPiece
        'GPT-2': AutoTokenizer.from_pretrained('gpt2'), # BPE
        'T5': AutoTokenizer.from_pretrained('t5-small'), # SentencePiece
        'RoBERTa': AutoTokenizer.from_pretrained('roberta-base'),

        # 'LLAMA': AutoTokenizer.from_pretrained('meta-llama/Llama-2-7b-hf'), # SentencePiece
        'BERT_MULTI': AutoTokenizer.from_pretrained('bert-base-multilingual-cased'), # 다국어 BERT
        'mBART': AutoTokenizer.from_pretrained('facebook/mbart-large-50'), # 다국어 지원
        "T5/mT5": AutoTokenizer.from_pretrained("google/mt5-small"),  # 다국어 지원
        'XLM-RoBERTa': AutoTokenizer.from_pretrained('xlm-roberta-base'), # 다국어 지원

        # 한국어 특화
        "KoBERT": AutoTokenizer.from_pretrained("skt/kobert-base-v1"),
        "KoGPT": AutoTokenizer.from_pretrained("skt/kogpt2-base-v2"),
        "KLUE": AutoTokenizer.from_pretrained("klue/roberta-base")
    }

    # GPT-2 토크나이저는 BOS/EOS 토큰 추가하지 않도록 설정
    tokenizers['GPT-2'].add_special_tokens({'pad_token': '[PAD]'})

    return tokenizers


In [56]:
import unicodedata as ucd # 한글 정규화 처리

def evaluate_tokenizers(tokenizer, texts, tokenizer_name=None):
    """
    토큰화 품질 평가 함수

    매개변수:
    - tokenizer: 평가할 토크나이저 객체
    - texts: 평가에 사용할 텍스트 리스트
    - tokenizer_name: 토크나이저 이름 (결과 출력용)

    반환값:
    - 평가 지표를 담은 딕셔너리
    """

    results = {}
    name = tokenizer_name
    print(f"\n===== {name} 토크나이저 평가 =====")

    # 분석을 위한 변수 초기화
    all_tokens = [] # 모든 텍스트 리스트의 토큰들
    token_lengths = [] # 각 토큰의 문자길이
    tokens_per_sentence = [] # 각 텍스트별 토큰수 모음
    unk_count = 0 # 모든 텍스트 리스트에서 나온 unk갯수의 합
    total_tokens = 0 # 모든 텍스트 리스트에서 나온 토큰수의 합
    recon_matches = 0 # 원본 문장과 디코딩된 문장이 문장이 같은 경우의 수

    # 토크나이저별 UNK 토큰 확인
    unk_token = tokenizer.unk_token
    print(f" {name} 토크나이저 UNK 토큰 : {unk_token}")

    # 각 텍스트별 분석
    for text in texts:
        # 토큰화
        tokens = tokenizer.tokenize(text)
        print(f'{name} 토큰: {tokens}')

        # 결과 저장
        all_tokens.extend(tokens)
        tokens_per_sentence.append(len(tokens))

        # UNK 토큰 개수 세기
        unk_count += tokens.count(unk_token)
        total_tokens += len(tokens)

        # 토큰 길이 계산 (특수 토큰 및 접두사 제외)
        # 자, 연, 어, 처, 리 vs 자연, 어, 처리
        # 접두사 제외 : ##어 - 1글자로 판단 (##, _ 제거)
        # 특수토큰 제외 : [UNK] : 5글자 - 글자 수 평균을 상승 왜곡 발생 ([, ], <, > 문자열 제거)
        for token in tokens:
            # 접두사 제거
            clean_token = token.replace("##", "") if "##" in token else token
            clean_token = clean_token.replace("_", "") if "_" in clean_token else clean_token
            # 특수 토큰 건너뛰기
            if not (clean_token.startswith("[") and clean_token.endswith("]")) and \
               not (clean_token.startswith("<") and clean_token.endswith(">")):
                token_lengths.append(len(clean_token))

        # 재구성 정확도 확인
        # 토큰화 후 다시 텍스트로 변환
        input_ids = tokenizer.encode(text)
        decoded = tokenizer.decode(input_ids)
        print(f'원본 텍스트: {text}')
        print(f'디코딩 결과: {decoded}')

        # 공백 정규화 후 비교 (연속된 공백, \n과 같은 문자열 제거)
        normalized_text = ucd.normalize('NFC', " ".join(text.split()))
        normalized_decoded = ucd.normalize('NFC', " ".join(decoded.split()))

        # print(f'원본 텍스트: {normalized_text}')
        # print(f'디코딩 텍스트: {normalized_decoded}')

        if normalized_text == normalized_decoded:
            recon_matches += 1

    # 1. UNK 토큰 빈도
    unk_ratio = unk_count / total_tokens if total_tokens > 0 else 0
    results["unk_ratio"] = unk_ratio

    # 2. 평균 토큰 길이
    avg_token_length = np.mean(token_lengths) if token_lengths else 0
    results["avg_token_length"] = avg_token_length

    # 3. 문장당 평균 토큰 수
    avg_tokens_per_sentence = np.mean(tokens_per_sentence)
    results["avg_tokens_per_sentence"] = avg_tokens_per_sentence

    # 4. 재구성 정확도
    recon_acc = recon_matches / len(texts)
    results["recon_accuracy"] = recon_acc

    # 결과 출력
    print(f"\n {name} 토크나이저 평가 결과 :")
    print(f"1. UNK 토큰 비율: {unk_ratio:.4f} ({unk_count}/{total_tokens})")
    print(f"2. 평균 토큰 길이: {avg_token_length:.2f}")
    print(f"3. 문장당 평균 토큰 수: {avg_tokens_per_sentence:.2f}")
    print(f"4. 재구성 정확도: {recon_acc:.4f} ({recon_matches}/{len(texts)})")

    return results

In [40]:
tokenizers = load_tokenizers()
texts = ['가장 별로라고?? 갤럭시 S25플러스 2주 실사용 장단점 리뷰']

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/531 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/752k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [57]:
for name, tokenizer in tokenizers.items():
  evaluate_tokenizers(tokenizer, texts, name)


===== BERT 토크나이저 평가 =====
 BERT 토크나이저 UNK 토큰 : [UNK]
BERT 토큰: ['ᄀ', '##ᅡ', '##ᄌ', '##ᅡ', '##ᆼ', 'ᄇ', '##ᅧ', '##ᆯ', '##ᄅ', '##ᅩ', '##ᄅ', '##ᅡ', '##ᄀ', '##ᅩ', '?', '?', 'ᄀ', '##ᅢ', '##ᆯ', '##ᄅ', '##ᅥ', '##ᆨ', '##ᄉ', '##ᅵ', 's', '##25', '##ᄑ', '##ᅳ', '##ᆯ', '##ᄅ', '##ᅥ', '##ᄉ', '##ᅳ', '2', '##ᄌ', '##ᅮ', 'ᄉ', '##ᅵ', '##ᆯ', '##ᄉ', '##ᅡ', '##ᄋ', '##ᅭ', '##ᆼ', 'ᄌ', '##ᅡ', '##ᆼ', '##ᄃ', '##ᅡ', '##ᆫ', '##ᄌ', '##ᅥ', '##ᆷ', 'ᄅ', '##ᅵ', '##ᄇ', '##ᅲ']
원본 텍스트: 가장 별로라고?? 갤럭시 S25플러스 2주 실사용 장단점 리뷰
디코딩 결과: [CLS] 가장 별로라고?? 갤럭시 s25플러스 2주 실사용 장단점 리뷰 [SEP]

 BERT 토크나이저 평가 결과 :
1. UNK 토큰 비율: 0.0000 (0/57)
2. 평균 토큰 길이: 1.02
3. 문장당 평균 토큰 수: 57.00
4. 재구성 정확도: 0.0000 (0/1)

===== GPT-2 토크나이저 평가 =====
 GPT-2 토크나이저 UNK 토큰 : <|endoftext|>
GPT-2 토큰: ['ê', '°', 'Ģ', 'ì', 'ŀ', '¥', 'Ġë', '³', 'Ħ', 'ë', '¡', 'ľ', 'ë', 'Ŀ', '¼', 'ê', '³', 'ł', '??', 'Ġ', 'ê', '°', '¤', 'ë', 'Ł', 'Ń', 'ì', 'ĭ', 'ľ', 'ĠS', '25', 'í', 'Ķ', 'Į', 'ë', 'Ł', '¬', 'ì', 'Ĭ', '¤', 'Ġ2', 'ì', '£', '¼', 'Ġì', 'ĭ', '¤

### 토큰화 정량적 성능 판단
1. UNK 처리 비중
2. 한 토큰의 길이(문자 수): 너무 짧게 잘리면 좋지않음
3. 토큰수: 적을수록 효율적
4. 재구조화시 복원률 계산: 인코딩-디코딩 -> 복원정도 여부